In [2]:
import numpy as np

from webbink import radius

## Calculating the ratio of bremsstrahlung to Compton cooling in a white dwarf boundary layer

We have claimed in the past that accretion disk boundary layers may be cooled by ambient radiation.  The code below allows us to calculate this under various physical conditions.  If electon cooling via bremsstrahlung is more efficient that cooling via inverse Compton scattering, then the radiation from the boundary layer will reflect the post-shock temperature of the gas in some way.  However, if Compton cooling is more efficient, then the maximum energy of electrons in the boundary layer will be decreased, and we don't expect to see hard X-rays characteristic of the post shock region.


Following Frank, King and Raine (2002), the ratio of bremsstrahlung to Compton lossed can be given by the equation:

$$\frac{t_{\rm Comp}}{t_{\rm cool}} = \frac{7.5 \times 10^{-5}N_{e}}{U_{\rm rad}T_{e}^{0.5}}$$ where $N_{e}$ and $T_{e}$ are the electron density and temperature in the boundary layer, and $U_{\rm rad}$ is the radiation energy density of the external radiation source that could be cooling the boundary layer (e.g. white dwarf surface, optically thick boundary layer).

First, we'll set up some global variables that are needed in the various calculations below:


In [17]:
MSOL = 2e33 # mass of Sun, g
MU = 0.63 #mean molecular weight
MH = 1.67e-24 # mass of hydrogen, g
G = 6.67e-8
K = 1.38e-16 # Boltzmann Constant, erg/K
KTOKEV = 8.6173e-8 # convert Kelvin to keV
C = 2e10 # speed of light, cm/s
MDOTGS = 6.34e+25 # convert mass accretion rate in msol/yr to g/s

The maximum temperature $T_{e}$ of the post-shock flow is our key observable. We first want to calculate what this would be if the temperature is simply set by the gravitational potential of the white dwarf.

In [15]:
def tshock(mwd, unit='K'):
    '''Returns the shock temperature for the innermost Keplerian orbit of an
    accretion disk around a white dwarf of mass mwd in solar units.
    Units can be Kelvin ('K') or keV ('keV').'''
    # mass in solar masses
    rwd = radius(mwd)
    mwdg = mwd * MSOL # mass of white dwarf in grams
    tshock = 3.0 * MU * MH * G * mwdg / (16.0 * K * rwd)
    if unit.lower() == 'k':
        return tshock
    elif unit.lower() == 'kev':
        return tshock * KTOKEV
    else:
        print("Units must be Kelvin (K) or keV (keV).  Please check your units.")
        
print("Shock temperature for 1.3 Msol white dwarf is {:03.1e} K.".format(tshock(1.3, unit='K')))
print("Shock temperature for 1.3 Msol white dwarf is {:03.1f} keV.".format(tshock(1.3, unit='keV')))

Shock temperature for 1.3 Msol white dwarf is 7.6e+08 K.
Shock temperature for 1.3 Msol white dwarf is 65.1 keV.


Next, we need a way to calculate the energy density of the radiation field, $U_{\rm rad}$, that might cool the post shock region. This is agnostic about what the source of the radiation is: we just need a luminosity (which we'll calculate elsewhere).

We assume that the source of radiation covers some fraction frad of the surface of a white dwarf of mass mwd (in solar units), so that
$$U_{\rm rad} = \frac{L_{\rm rad}}{4 \pi R_{\rm WD}^{2} c},$$ where $c$ is the speed of light.

In [16]:
def urad(lrad, mwd, frad):
    '''Returns the radiation energy density for a radiation source of 
    luminosity Lrad (in erg/s) assuming it covers a fraction frad of
    the surface of a white dwarf of mass mwd.
    Mass should be in solar units.'''
    if frad <= 0 or frad > 1:
        print("frad must be between 0 and 1. Please check and try again.")
    else:
        rwd = radius(mwd)
        urad = lrad / (4.0 * np.pi * rwd ** 2.0 * frad * C)
        return urad

The other key parameter in determining the cooling ratio is the electron density in the boundary layer. Assuming a constant mass accretion rate through the boundary layer, we can calculate this as 
$$N_{e} = \frac{\dot{M}}{4 \pi R_{\rm WD}^{2} \mu m_{\rm H} f_{\rm acc} v_{\rm acc}},$$ where $f_{\rm acc}$ is the fraction of the white dwarf surface over which the accretion takes place, and $v_{\rm acc}$ is the radial velocity with which material is moving through the boundary layer.

The accretion speed through the boundary layer is not well known. The innermost Keplerian velocity of the disk, $v_{\rm Kep}$, is given by
$$v_{\rm Kep} = \sqrt{\frac{G M_{\rm WD}}{R_{\rm WD}}}.$$

In [18]:
def vkep(mwd):
    rwd = radius(mwd)
    mwdg = mwd * MSOL
    vkep = (G * mwdg / rwd) ** 0.5
    return vkep

How much of this circular velocity gets transferred into radial velocity? Explore claims here.  For now, we assume that the accretion velocity, $v_{\rm acc}$ is low, of order 10 km s$^{-1}$.

In [19]:
vacc = 10e5

What about $f_{\rm acc}$, the fraction of the white dwarf surface over which the accretion takes place?  This depends on the assumed structure of the boundary layer, and whether it remains optically thick or thin.  Several approximations of this exist in the literature.

In [20]:
def facc(mwd, mdot, mode="thick"):
    '''Returns an estimte of the accretion fraction facc for a thick or thin
    boundary layer.  Facc is calculated as z/4r, where z is the scale height
    of disk.  If mode = "thick", the disk scale height is assumed to be equal 
    to the disk scale height. If mode = "thin", use approximation of Tylenda
    (1981).  See Patterson & Raymond, 1985 for details.'''
    if mode.lower() == "thick":
        z = (np.sqrt(radius(mwd)) ** 2.0 * \
            (6.96e-4 * (mwd/0.7) ** -0.85 * (mdot * MDOTGS / 1e18) ** -0.22 + \
            (7.29e-4 * (mwd/0.7) ** 0.8 * (mdot * MDOTGS / 1e18))))
        facc = z / 4 / radius(mwd)
        return facc
    elif mode.lower() == "thin":
        z = 3.85e8 * (mwd/0.7) ** 0.1 * (mdot * MDOTGS / 1e15) ** -0.5
        facc = z / 4 / radius(mwd)
        return facc
    else:
        print("mode must be thick or thin")